In [14]:
def HPS(r, rmin, rmax, eps, lambd, sigma):
    V = 4*eps*((sigma/r)**12 - (sigma/r)**6)
    F = 4*eps/r*(12*(sigma/r)**12 - 6*(sigma/r)**6)
#     if r <= 2**(1/6)*sigma:
#         V = V + (1-lambd)*eps
#     else:
#         V = lambd*V
#         F = lambd*F
    return (V, F)

In [15]:
pos = [[ 1.04550171e+01, -2.48057175e+01,  4.45167542e+00],
 [ 1.33750153e+01, -2.24057198e+01,  4.08166885e+00],
 [ 1.51950073e+01, -1.93357277e+01,  2.72167206e+00],
 [ 1.46150208e+01, -1.89757156e+01, -1.04831696e+00],
 [ 1.13050079e+01, -2.09957314e+01,  1.01680756e-01],
 [ 9.78500366e+00, -2.37457275e+01, -2.64831543e+00],
 [ 6.74499512e+00, -2.39657211e+01, -4.15832520e+00],
 [ 4.56501007e+00, -2.16657257e+01, -5.96832275e+00],
 [ 1.75502777e+00, -2.18557167e+01, -8.87832642e+00],
 [ 6.25000000e-01, -2.53757286e+01, -8.61831665e+00],
 [ 2.88501740e+00, -2.82757187e+01, -7.39831924e+00],
 [ 6.45027161e-01, -3.11957169e+01, -6.87831879e+00],
 [ 1.29501343e+00, -3.27657318e+01, -3.39832306e+00],
 [ 9.25025940e-01, -2.90357208e+01, -1.75832748e+00],
 [ 1.27502441e+00, -2.82757187e+01,  1.92167282e+00],
 [-5.64994812e-01, -2.75357246e+01,  5.03168106e+00],
 [ 7.05032349e-01, -2.43757248e+01,  6.94168091e+00],
 [ 2.99499512e+00, -2.11757278e+01,  5.55168152e+00],
 [ 2.88501740e+00, -2.04357147e+01,  2.03166962e+00],
 [ 5.93502045e+00, -1.99857330e+01, -2.58331299e-01],
 [ 6.82502747e+00, -1.64957237e+01,  7.11669922e-01],
 [ 8.92501831e+00, -1.41657257e+01,  3.09167862e+00],
 [ 1.27550125e+01, -1.39157295e+01,  4.09168243e+00],
 [ 1.46950150e+01, -1.06157303e+01,  3.24167252e+00],
 [ 1.19750214e+01, -8.37572098e+00,  1.84167862e+00],
 [ 8.27503204e+00, -7.46572495e+00,  7.51686096e-01],
 [ 5.70503235e+00, -8.71572495e+00,  3.23167801e+00],
 [ 7.23503113e+00, -9.97571945e+00,  6.50167465e+00],
 [ 1.02149963e+01, -8.54572296e+00,  8.17167282e+00],
 [ 1.01650238e+01, -1.00257301e+01,  1.19516754e+01],
 [ 6.84501648e+00, -1.02257156e+01,  1.36816788e+01],
 [ 4.68502045e+00, -1.04557228e+01,  1.66216850e+01],
 [ 5.34500122e+00, -1.19557190e+01,  1.99516869e+01],
 [ 4.53502655e+00, -1.28057289e+01,  2.34316826e+01],
 [ 6.97502136e+00, -1.59757233e+01,  2.43916702e+01],
 [ 9.72499847e+00, -1.57857323e+01,  2.18216705e+01],
 [ 1.28250122e+01, -1.67557144e+01,  2.40816689e+01],
 [ 1.44350052e+01, -1.50857162e+01,  2.68216705e+01],
 [ 1.54850006e+01, -1.14757156e+01,  2.54116821e+01],
 [ 1.14750290e+01, -1.00657272e+01,  2.50216866e+01],
 [ 1.22150040e+01, -7.66572952e+00,  2.78516769e+01],
 [ 9.19502258e+00, -8.53572845e+00,  3.05316734e+01],
 [ 7.12501526e+00, -7.35572815e+00,  3.36016846e+01],
 [ 5.49499512e+00, -4.52571869e+00,  3.56716728e+01],
 [ 2.04502106e+00, -5.48572540e+00,  3.57516861e+01],
 [ 2.50499725e+00, -8.97571564e+00,  3.75516701e+01],
 [ 5.07499695e+00, -1.16657257e+01,  3.63116837e+01],
 [ 3.92501831e+00, -1.44057274e+01,  3.39616776e+01],
 [ 1.05018616e-01, -1.34357262e+01,  3.29816818e+01],
 [-2.32498169e+00, -1.41257286e+01,  3.06716728e+01],
 [-5.05496979e+00, -1.40857315e+01,  2.75916862e+01],
 [-4.18498993e+00, -1.43457222e+01,  2.35916710e+01],
 [-3.76499176e+00, -1.74257278e+01,  2.12116814e+01],
 [-8.74977112e-01, -1.84757233e+01,  1.90716743e+01],
 [-1.15497589e+00, -1.63557243e+01,  1.63316727e+01],
 [-3.34968567e-01, -1.81257248e+01,  1.32416725e+01],
 [ 2.45018005e-01, -1.71157265e+01,  9.78168488e+00],
 [-2.04498291e+00, -1.92357254e+01,  7.37167358e+00],
 [-4.50500488e+00, -2.18257141e+01,  7.31168747e+00],
 [-8.16497803e+00, -2.16757202e+01,  6.08167648e+00],
 [-9.95498657e+00, -1.93857193e+01,  3.42166901e+00],
 [-9.34497833e+00, -1.66157150e+01,  8.61682892e-01],
 [-5.63499451e+00, -1.63757324e+01,  2.51674652e-01],
 [-4.89498138e+00, -1.91257286e+01, -2.27832794e+00],
 [-1.65496826e+00, -2.07257271e+01, -1.58329010e-01],
 [-2.30499268e+00, -2.32457161e+01, -3.28832626e+00],
 [-4.58499908e+00, -2.37357330e+01, -6.35831833e+00],
 [-4.81498718e+00, -2.17557144e+01, -9.77832794e+00],
 [-2.94498444e+00, -2.08657265e+01, -1.28783226e+01],
 [-1.66500092e+00, -1.83657265e+01, -1.53483200e+01],
 [-1.30496979e+00, -1.45757294e+01, -1.37583160e+01],
 [ 2.65007019e-01, -1.53657150e+01, -1.05083275e+01],
 [ 3.20503235e+00, -1.40757179e+01, -8.21832657e+00],
 [ 1.77501678e+00, -1.26757240e+01, -5.02832413e+00],
 [ 2.19501495e+00, -8.95572662e+00, -5.74832916e+00],
 [ 2.45502472e+00, -5.37572861e+00, -4.82831955e+00],
 [ 3.95011902e-01, -3.27571869e+00, -2.35832214e+00],
 [-2.06497192e+00, -3.97571564e+00,  3.21674347e-01],
 [-1.88499451e+00, -7.44571686e+00,  5.61676025e-01],
 [-4.02500153e+00, -1.02957153e+01,  2.36167908e+00],
 [-4.46498871e+00, -1.10657310e+01,  6.34168625e+00],
 [-2.00500488e+00, -1.16357231e+01,  9.37168121e+00],
 [ 2.55012512e-01, -8.51572037e+00,  9.80167389e+00],
 [ 1.49501801e+00, -5.84571838e+00,  1.22716713e+01],
 [ 4.22500610e+00, -3.96572113e+00,  1.05616760e+01],
 [ 3.63502502e+00, -4.25720215e-01,  9.54168320e+00],
 [ 6.15501404e+00,  2.05427170e+00,  1.19016838e+01],
 [ 1.00049973e+01,  1.75428391e+00,  1.10416794e+01],
 [ 1.12850189e+01, -2.75726318e-01,  7.52168655e+00],
 [ 1.48450089e+01,  9.14268494e-01,  6.56167984e+00],
 [ 1.60850143e+01,  4.19427872e+00,  8.19168091e+00],
 [ 1.61149979e+01,  3.71427536e+00,  1.19216728e+01],
 [ 1.85050201e+01,  3.95427704e+00,  1.47416687e+01],
 [ 2.21650314e+01,  3.41426849e+00,  1.45116806e+01],
 [ 2.42350006e+01,  2.35427856e+00,  1.75316811e+01],
 [ 2.42150116e+01, -1.55715942e-01,  2.04416847e+01],
 [ 2.05450058e+01, -1.04572296e+00,  2.09916687e+01],
 [ 1.72650146e+01,  7.14282990e-01,  2.10016823e+01],
 [ 1.50150299e+01,  3.94428253e+00,  2.05216789e+01],
 [ 1.12950134e+01,  3.73428345e+00,  1.90816689e+01],
 [ 8.34503174e+00,  1.29426956e+00,  1.94616699e+01],
 [ 9.29500580e+00, -5.05714417e-01,  2.28316689e+01],
 [ 9.44499969e+00, -3.80573273e+00,  2.10216713e+01],
 [ 7.52502441e+00, -7.14572906e+00,  2.03216743e+01],
 [ 4.42501068e+00, -5.97572327e+00,  2.16016769e+01],
 [ 3.25500488e+00, -5.43571472e+00,  2.53416824e+01],
 [-2.84996033e-01, -5.65572739e+00,  2.36516762e+01],
 [-5.14984131e-01, -6.96573257e+00,  1.99616814e+01],
 [-3.47499847e+00, -8.35573196e+00,  1.73416710e+01],
 [-2.83496857e+00, -1.14657211e+01,  1.53216743e+01],
 [-5.63499451e+00, -1.31457329e+01,  1.37116814e+01],
 [-7.72499084e+00, -1.40457153e+01,  1.69616699e+01],
 [-9.23500061e+00, -1.12357330e+01,  1.93816757e+01],
 [-1.25749969e+01, -1.11257172e+01,  2.07616806e+01],
 [-1.51249695e+01, -1.31257248e+01,  1.86816788e+01],
 [-1.83250046e+01, -1.54857254e+01,  1.81616783e+01],
 [-1.94549942e+01, -1.73157310e+01,  1.51216698e+01],
 [-2.24049759e+01, -1.83157158e+01,  1.28416824e+01],
 [-2.20750046e+01, -2.17557144e+01,  1.11916733e+01],
 [-2.33049774e+01, -2.06857300e+01,  7.37167358e+00],
 [-2.66049957e+01, -1.90957260e+01,  7.44167328e+00],
 [-2.76149750e+01, -2.01857185e+01,  1.09916687e+01],
 [-3.13649750e+01, -1.97757149e+01,  1.10916710e+01],
 [-3.28049850e+01, -2.31057167e+01,  1.27516747e+01],
 [-3.64449692e+01, -2.35157204e+01,  1.37616730e+01],
 [-3.83449936e+01, -2.22157288e+01,  1.10016823e+01],
 [-3.87949753e+01, -1.81757164e+01,  1.14916801e+01],
 [-3.74149704e+01, -1.77657318e+01,  1.50916862e+01],
 [-3.97749710e+01, -1.65457153e+01,  1.76716805e+01],
 [-4.19949722e+01, -1.39357185e+01,  1.70716858e+01],
 [-4.16849899e+01, -1.21457291e+01,  2.04916763e+01],
 [-3.79049683e+01, -1.13157272e+01,  2.15816689e+01],
 [-3.43949890e+01, -1.15557289e+01,  2.29916763e+01],
 [-3.10649872e+01, -1.00457191e+01,  2.23316765e+01],
 [-3.02549744e+01, -6.91572189e+00,  2.40216827e+01],
 [-2.84349823e+01, -3.83571625e+00,  2.29016685e+01],
 [-2.52949905e+01, -2.03573227e+00,  2.36316872e+01],
 [-2.20649719e+01, -1.10572815e+00,  2.14716721e+01],
 [-1.99250031e+01, -4.58572388e+00,  2.08616829e+01],
 [-1.64749908e+01, -5.18571854e+00,  2.10316849e+01],
 [-1.29449844e+01, -5.71573257e+00,  2.07316780e+01],
 [-1.14449692e+01, -4.39573288e+00,  2.39616776e+01],
 [-1.17549896e+01, -3.63573074e+00,  2.75616837e+01],
 [-1.54549789e+01, -2.55573273e+00,  2.72716713e+01],
 [-1.65449905e+01,  1.23428345e+00,  2.77416801e+01],
 [-1.83849716e+01,  2.47426987e+00,  3.09416771e+01],
 [-2.19549942e+01,  3.12427521e+00,  3.10716820e+01],
 [-2.35350037e+01,  7.54280090e-01,  2.79816818e+01],
 [-2.61349869e+01, -1.75571442e+00,  2.92716789e+01],
 [-2.46549988e+01, -5.40573120e+00,  2.91916847e+01],
 [-2.09149933e+01, -4.71572876e+00,  3.03316689e+01],
 [-2.12249756e+01, -8.43572617e+00,  2.87716866e+01],
 [-2.34249878e+01, -8.43572617e+00,  2.54816818e+01],
 [-2.36249924e+01, -1.21557236e+01,  2.63416862e+01],
 [-2.53549957e+01, -1.40257263e+01,  2.35116768e+01],
 [-2.43849945e+01, -1.20757294e+01,  2.02416801e+01],
 [-2.26849747e+01, -1.08157158e+01,  1.70916748e+01],
 [-2.03949738e+01, -1.13857269e+01,  1.41916847e+01],
 [-1.95449829e+01, -8.07571411e+00,  1.31816864e+01],
 [-2.23649979e+01, -6.16571426e+00,  1.44916725e+01],
 [-2.51350021e+01, -3.58572006e+00,  1.48216820e+01],
 [-2.88150024e+01, -3.10571671e+00,  1.65416718e+01],
 [-3.02849960e+01, -6.43571854e+00,  1.48716736e+01],
 [-2.82049942e+01, -6.96573257e+00,  1.17216873e+01],
 [-2.54649734e+01, -8.01572800e+00,  8.78168106e+00],
 [-2.73550034e+01, -9.85572815e+00,  6.53167725e+00],
 [-2.99349976e+01, -9.57572937e+00,  3.47167969e+00],
 [-3.37349701e+01, -1.02957153e+01,  3.61167908e+00],
 [-3.57949829e+01, -1.09757233e+01,  6.66168213e+00],
 [-3.77649689e+01, -8.39572906e+00,  8.23167801e+00],
 [-4.05949783e+01, -9.03572083e+00,  1.06416702e+01],
 [-3.94849777e+01, -7.86571503e+00,  1.41416740e+01],
 [-4.08549881e+01, -5.11571884e+00,  1.67316818e+01],
 [-4.35949707e+01, -4.48572159e+00,  1.40816689e+01],
 [-4.68349838e+01, -2.66572952e+00,  1.43016815e+01],
 [-4.65950012e+01, -9.05723572e-01,  1.09816742e+01],
 [-4.30449677e+01, -1.82571411e+00,  9.59167480e+00],
 [-3.97249985e+01, -3.29572678e+00,  1.00816727e+01],
 [-3.81449890e+01,  3.42750549e-02,  1.03916740e+01],
 [-3.73949814e+01,  2.71427155e+00,  1.29416847e+01],
 [-3.78149796e+01,  5.74428558e+00,  1.10816765e+01],
 [-3.43349838e+01,  6.74427032e+00,  1.01116753e+01],
 [-3.26549911e+01,  4.65427399e+00,  7.12167740e+00],
 [-2.91949844e+01,  3.96427155e+00,  5.52167892e+00],
 [-2.53049850e+01,  2.78427124e+00,  6.20168686e+00],
 [-2.28249741e+01,  5.34267426e-01,  4.16168213e+00],
 [-2.40149689e+01, -3.06571960e+00,  5.47168732e+00],
 [-2.16249847e+01, -3.78572464e+00,  8.32168579e+00],
 [-1.84949875e+01, -1.58573151e+00,  7.93167114e+00],
 [-1.57349777e+01, -4.11571503e+00,  9.29168701e+00],
 [-1.37549973e+01, -7.47571945e+00,  9.72167969e+00],
 [-1.02949905e+01, -8.92572403e+00,  1.07916832e+01],
 [-1.00449753e+01, -9.43572998e+00,  1.46016693e+01],
 [-1.04649734e+01, -5.68572998e+00,  1.54116821e+01],
 [-9.82498169e+00, -3.05572510e+00,  1.23216820e+01],
 [-6.85497284e+00, -1.79573059e+00,  1.42816734e+01],
 [-3.07498932e+00, -1.58573151e+00,  1.38516808e+01],
 [-2.05001831e-01, -2.20571518e+00,  1.61516762e+01],
 [ 1.04499817e+00, -6.55727386e-01,  1.93816757e+01],
 [ 1.38500214e+00,  2.77427673e+00,  1.79016685e+01],
 [-6.14967346e-01,  3.63428116e+00,  1.51516724e+01],
 [-3.65497589e+00,  4.96427536e+00,  1.27016830e+01],
 [-5.54496765e+00,  7.53427505e+00,  1.43116760e+01],
 [-5.09498596e+00,  1.04842758e+01,  1.21216774e+01],
 [-1.70497894e+00,  1.07142830e+01,  1.01316833e+01],
 [-1.72496796e+00,  1.46342850e+01,  1.07916832e+01],
 [ 6.45027161e-01,  1.63542747e+01,  1.28216743e+01],
 [ 4.27501678e+00,  1.71442795e+01,  1.13316727e+01],
 [ 3.72501373e+00,  1.59942818e+01,  7.57167816e+00],
 [ 3.35502625e+00,  1.87442780e+01,  4.91167068e+00],
 [ 1.87500000e+00,  2.14242744e+01,  6.81167603e+00],
 [ 1.34502411e+00,  2.48642731e+01,  5.91167450e+00],
 [-4.15000916e-01,  2.73042679e+01,  8.26168060e+00],
 [-3.91498566e+00,  2.70142746e+01,  7.48167038e+00],
 [-5.67497253e+00,  3.03342819e+01,  6.64167404e+00],
 [-7.88497925e+00,  3.37442780e+01,  7.01168060e+00],
 [-6.30496979e+00,  3.63142776e+01,  4.54168320e+00],
 [-4.83497620e+00,  3.81742859e+01,  2.04168320e+00],
 [-7.59498596e+00,  3.89242744e+01, -9.68322754e-01],
 [-8.93497467e+00,  4.11942673e+01, -3.55833054e+00],
 [-8.54499817e+00,  4.51142693e+01, -4.34831619e+00],
 [-1.09249878e+01,  4.83042717e+01, -4.45831299e+00],
 [-1.26449966e+01,  5.15042686e+01, -4.82831955e+00],
 [-1.40649796e+01,  5.52642822e+01, -4.33832169e+00],
 [-1.21349716e+01,  5.67042732e+01, -7.63832092e+00],
 [-9.86499786e+00,  5.50442696e+01, -9.89831924e+00],
 [-1.02649689e+01,  5.39742851e+01, -1.36083221e+01],
 [-1.29850006e+01,  5.53742790e+01, -1.53883171e+01],
 [-1.63949966e+01,  5.47342682e+01, -1.70683289e+01],
 [-1.85449982e+01,  5.19242859e+01, -1.62883186e+01],
 [-1.75849915e+01,  4.87742805e+01, -1.45083237e+01],
 [-1.39149857e+01,  4.87442780e+01, -1.49583244e+01],
 [-1.25749969e+01,  4.52942848e+01, -1.52583313e+01],
 [-1.42449951e+01,  4.18542671e+01, -1.54083252e+01],
 [-1.19949722e+01,  3.94542694e+01, -1.33583260e+01],
 [-9.81498718e+00,  3.65742683e+01, -1.28283310e+01],
 [-7.35500336e+00,  3.40942764e+01, -1.33283234e+01],
 [-6.02497101e+00,  3.15842819e+01, -1.05183220e+01],
 [-2.33497620e+00,  3.12542725e+01, -1.12683296e+01],
 [ 7.05032349e-01,  3.07642746e+01, -1.37283134e+01],
 [ 1.61502838e+00,  3.05942726e+01, -1.71983147e+01],
 [ 3.80500793e+00,  3.31142807e+01, -1.88483238e+01],
 [ 6.87500000e+00,  3.45742798e+01, -1.81183243e+01],
 [ 1.05450058e+01,  3.42542839e+01, -1.86383247e+01],
 [ 9.82501984e+00,  3.07342720e+01, -2.05283165e+01],
 [ 1.12350082e+01,  2.75542831e+01, -1.91083145e+01],
 [ 1.16250229e+01,  2.37242699e+01, -1.94083214e+01],
 [ 1.18950272e+01,  1.98942757e+01, -2.01183128e+01],
 [ 1.38450241e+01,  1.91342735e+01, -2.31683159e+01],
 [ 1.40050125e+01,  1.87542725e+01, -2.71183205e+01],
 [ 1.22350311e+01,  2.06442833e+01, -2.98783302e+01],
 [ 8.57501984e+00,  2.01042747e+01, -3.03283310e+01],
 [ 5.39501190e+00,  2.18942833e+01, -3.05883217e+01],
 [ 3.57501984e+00,  2.40542793e+01, -2.81383133e+01],
 [ 4.84500885e+00,  2.61742783e+01, -2.49883270e+01],
 [ 4.21501160e+00,  2.36942673e+01, -2.18983269e+01],
 [ 4.96501923e+00,  2.67542839e+01, -1.99783134e+01],
 [ 5.86502075e+00,  2.51242828e+01, -1.64083290e+01],
 [ 8.35502625e+00,  2.28942680e+01, -1.43883133e+01],
 [ 6.75502777e+00,  2.01942825e+01, -1.72583199e+01],
 [ 5.83499908e+00,  1.68942833e+01, -1.87883186e+01],
 [ 7.61501312e+00,  1.60642815e+01, -2.17483139e+01],
 [ 9.07501221e+00,  1.23042679e+01, -2.16783142e+01],
 [ 9.19502258e+00,  8.62426758e+00, -2.10983276e+01],
 [ 8.38500977e+00,  4.82427597e+00, -2.03083229e+01],
 [ 7.55500793e+00,  4.80426788e+00, -1.67383194e+01],
 [ 5.87501526e+00,  8.06427002e+00, -1.65683174e+01],
 [ 6.78501129e+00,  1.16542816e+01, -1.51183128e+01],
 [ 9.71500397e+00,  1.39442825e+01, -1.52083206e+01],
 [ 8.33499908e+00,  1.59542847e+01, -1.20983315e+01],
 [ 5.99502563e+00,  1.46842766e+01, -9.36832428e+00],
 [ 4.12502289e+00,  1.64542770e+01, -6.73831940e+00],
 [ 3.23501587e+00,  1.97342682e+01, -4.85832214e+00],
 [ 3.81500244e+00,  2.29042816e+01, -6.60831451e+00],
 [ 7.02503204e+00,  2.33542824e+01, -8.34833145e+00],
 [ 9.31499481e+00,  2.34942818e+01, -5.28831482e+00],
 [ 1.21950150e+01,  2.51842690e+01, -4.30831909e+00],
 [ 1.31350327e+01,  2.90042686e+01, -3.29832077e+00],
 [ 1.68450165e+01,  2.98242760e+01, -5.57832718e+00],
 [ 1.73649979e+01,  3.33642769e+01, -5.80831528e+00],
 [ 2.09150314e+01,  3.33742714e+01, -7.18832016e+00],
 [ 2.12950134e+01,  3.12042809e+01, -1.02783203e+01],
 [ 1.95250320e+01,  3.32042694e+01, -1.27983284e+01],
 [ 1.60950089e+01,  3.14642715e+01, -1.34683228e+01],
 [ 1.23750305e+01,  3.02642822e+01, -1.28983307e+01],
 [ 1.14250183e+01,  2.73042679e+01, -1.03383255e+01],
 [ 1.36650085e+01,  2.41442680e+01, -1.02083206e+01],
 [ 1.70250320e+01,  2.28642845e+01, -1.14483261e+01],
 [ 1.59550095e+01,  1.93442726e+01, -1.05383301e+01],
 [ 1.36849976e+01,  1.66142845e+01, -8.98832321e+00],
 [ 1.31949997e+01,  1.70742798e+01, -5.26832581e+00],
 [ 1.25749969e+01,  1.69042778e+01, -1.23832703e+00],
 [ 1.46450043e+01,  1.36242676e+01, -6.48326874e-01],
 [ 1.62750244e+01,  9.99427795e+00, -1.28831863e+00],
 [ 1.86350250e+01,  8.11428070e+00, -3.04832458e+00],
 [ 1.75450134e+01,  4.85427856e+00, -5.35831451e+00],
 [ 1.61950302e+01,  1.30428314e+00, -5.42831421e+00],
 [ 1.85050201e+01, -8.75720978e-01, -3.33831787e+00],
 [ 2.09049988e+01, -3.39572906e+00, -2.84832001e+00],
 [ 2.45750046e+01, -3.13571930e+00, -2.48832703e+00],
 [ 2.82250214e+01, -1.39572144e+00, -1.69832230e+00],
 [ 3.08749771e+01, -3.65715027e-01,  1.26167297e+00],
 [ 3.28550339e+01,  3.74279022e-01, -1.80831909e+00],
 [ 3.45750046e+01,  3.34426880e+00, -3.42832565e+00],
 [ 3.77650070e+01,  1.24427795e+00, -3.18832397e+00],
 [ 4.02750015e+01, -1.48572922e+00, -2.52832413e+00],
 [ 3.91550064e+01, -3.21573257e+00, -5.71832657e+00],
 [ 3.78650284e+01, -6.71571732e+00, -4.62831497e+00],
 [ 3.88450241e+01, -1.05557251e+01, -4.29832458e+00],
 [ 3.89249802e+01, -8.83571625e+00, -7.68318176e-01],
 [ 3.67850113e+01, -1.05857277e+01,  1.24168396e+00],
 [ 3.55150223e+01, -8.40572357e+00,  3.87166977e+00],
 [ 3.48050308e+01, -7.87572861e+00,  7.45168686e+00],
 [ 3.17150497e+01, -1.03157234e+01,  6.93168640e+00],
 [ 2.86250305e+01, -1.22357178e+01,  7.14168549e+00],
 [ 2.94649887e+01, -1.38757324e+01,  3.56168747e+00],
 [ 2.87450027e+01, -1.16057205e+01,  5.31673431e-01],
 [ 2.61650085e+01, -1.31657219e+01, -1.93832397e+00],
 [ 2.28750229e+01, -1.18557167e+01, -3.27831268e+00],
 [ 2.11250305e+01, -1.54357147e+01, -4.34831619e+00],
 [ 1.96350098e+01, -1.91357231e+01, -4.91832733e+00],
 [ 2.06650162e+01, -2.07657242e+01, -8.32832336e+00],
 [ 1.91850281e+01, -1.95557213e+01, -1.14883232e+01],
 [ 1.69950104e+01, -2.31057167e+01, -1.17583275e+01],
 [ 1.35050201e+01, -2.38557243e+01, -1.12383270e+01],
 [ 1.12450027e+01, -2.08157158e+01, -1.09383202e+01],
 [ 9.22500610e+00, -1.74457169e+01, -1.04683304e+01],
 [ 1.16049957e+01, -1.58657265e+01, -8.30831528e+00],
 [ 1.46450043e+01, -1.40457153e+01, -7.48832703e+00],
 [ 1.46650314e+01, -1.09457207e+01, -9.49832916e+00],
 [ 1.29249954e+01, -8.18572998e+00, -1.16983223e+01],
 [ 1.31550217e+01, -9.21571732e+00, -1.53083229e+01],
 [ 1.48850250e+01, -6.24572754e+00, -1.69083214e+01],
 [ 1.80850220e+01, -5.62572479e+00, -1.80683136e+01],
 [ 1.98450089e+01, -3.56573105e+00, -1.57583237e+01],
 [ 2.29450226e+01, -1.20573044e+00, -1.49383163e+01],
 [ 2.54050064e+01, -1.33571625e+00, -1.21883202e+01],
 [ 2.71750259e+01,  1.44271851e-01, -9.58831787e+00],
 [ 2.82250214e+01, -2.92572021e+00, -7.43831635e+00],
 [ 2.95250320e+01, -5.12573242e+00, -1.02283287e+01],
 [ 2.71450043e+01, -7.58571625e+00, -1.06383133e+01],
 [ 2.44150162e+01, -7.05572128e+00, -8.09831619e+00],
 [ 2.12350082e+01, -8.20571899e+00, -9.82831955e+00],
 [ 1.91150284e+01, -9.92572784e+00, -1.27183151e+01],
 [ 1.81150055e+01, -1.34257317e+01, -1.29883194e+01],
 [ 1.91749954e+01, -1.46157265e+01, -1.63683128e+01],
 [ 1.56650162e+01, -1.41857147e+01, -1.86483192e+01],
 [ 1.23250198e+01, -1.56357193e+01, -1.92483139e+01],
 [ 1.31850052e+01, -1.78857231e+01, -2.22983170e+01],
 [ 1.25250244e+01, -2.16657257e+01, -2.18383217e+01],
 [ 9.13501740e+00, -2.05457306e+01, -2.03083229e+01],
 [ 5.84503174e+00, -1.87857246e+01, -1.96183205e+01],
 [ 4.03499603e+00, -1.68157196e+01, -1.73183250e+01],
 [ 4.81502533e+00, -1.39057159e+01, -1.98883247e+01],
 [ 4.23500061e+00, -1.02857208e+01, -1.92083168e+01],
 [ 2.34500885e+00, -7.04572678e+00, -1.91983223e+01],
 [ 2.86502838e+00, -3.18572998e+00, -1.89183235e+01],
 [ 6.01501465e+00, -1.92571640e+00, -1.71283150e+01],
 [ 9.38503265e+00, -2.15572357e+00, -1.87583160e+01],
 [ 1.25550079e+01, -1.95732117e-01, -1.93783188e+01],
 [ 1.60350037e+01,  4.14276123e-01, -1.87183189e+01],
 [ 1.66450119e+01,  4.37427521e+00, -1.85283279e+01],
 [ 1.83650208e+01,  4.77428436e+00, -2.17283249e+01],
 [ 1.71050262e+01,  1.78426743e+00, -2.35883141e+01],
 [ 1.53549957e+01,  3.30427170e+00, -2.65783310e+01],
 [ 1.60850143e+01,  3.33427429e+00, -3.01583290e+01],
 [ 1.42449951e+01,  5.50428391e+00, -3.30583191e+01],
 [ 1.10750198e+01,  3.24428558e+00, -3.36083221e+01],
 [ 8.33499908e+00,  5.84278107e-01, -3.23683167e+01],
 [ 9.82501984e+00, -2.12572098e+00, -3.43183136e+01],
 [ 1.28350067e+01, -4.15573120e+00, -3.42183304e+01],
 [ 1.42449951e+01, -6.44573212e+00, -3.12883186e+01],
 [ 1.56450272e+01, -3.78572464e+00, -2.88183212e+01],
 [ 1.52050018e+01, -4.03572083e+00, -2.49083138e+01],
 [ 1.21750259e+01, -6.59572601e+00, -2.36483192e+01],
 [ 8.29502106e+00, -6.51573181e+00, -2.26683235e+01],
 [ 6.02500916e+00, -5.59572220e+00, -2.59183311e+01],
 [ 7.22499847e+00, -2.05572128e+00, -2.73883247e+01],
 [ 4.36500549e+00,  2.34279633e-01, -2.75583267e+01],
 [ 3.69503021e+00,  2.46427536e+00, -2.46583176e+01],
 [ 2.65502930e+00,  2.43427277e+00, -2.10683250e+01],
 [-5.44967651e-01,  1.78426743e+00, -1.94683266e+01],
 [-3.24497223e+00,  3.41426849e+00, -1.69183159e+01],
 [-4.21497345e+00,  2.31428146e+00, -1.33983231e+01],
 [-3.01498413e+00,  5.67428589e+00, -1.13283157e+01],
 [ 7.64999390e-01,  6.83427811e+00, -1.06483269e+01],
 [ 2.70500183e+00,  9.64427948e+00, -1.05383301e+01],
 [ 1.81499481e+00,  1.20342827e+01, -1.31983185e+01],
 [-1.35002136e-01,  1.08242798e+01, -1.64983177e+01],
 [-8.64982605e-01,  9.12427902e+00, -1.98883247e+01],
 [-3.96499634e+00,  1.07142830e+01, -2.12683296e+01],
 [-3.64498138e+00,  1.42242813e+01, -2.26983261e+01],
 [-4.85000610e-01,  1.61542702e+01, -2.17483139e+01],
 [-8.25004578e-01,  1.73742676e+01, -1.82183266e+01],
 [ 4.55017090e-01,  2.08242798e+01, -1.66083145e+01],
 [ 8.65020752e-01,  1.95442772e+01, -1.30783272e+01],
 [-2.48497009e+00,  1.86742783e+01, -1.11083221e+01],
 [-3.69499207e+00,  2.17042732e+01, -8.91832352e+00],
 [-4.80499268e+00,  2.01642799e+01, -5.39833069e+00],
 [-7.12497711e+00,  1.76442719e+01, -3.77832413e+00],
 [-6.59500122e+00,  1.41842842e+01, -5.59831619e+00],
 [-3.50498199e+00,  1.20242691e+01, -6.16832733e+00],
 [-1.73500061e+00,  1.31642723e+01, -3.05831909e+00],
 [ 4.99725342e-03,  1.11142731e+01, -3.98330688e-01],
 [ 3.29502106e+00,  1.33342743e+01, -9.18331146e-01],
 [ 5.93502045e+00,  1.15642738e+01, -3.57831955e+00],
 [ 3.98502350e+00,  8.33427429e+00, -4.21833038e+00],
 [ 4.01500702e+00,  5.66427231e+00, -1.82832718e+00],
 [ 5.56499481e+00,  2.80427933e+00,  8.16726685e-02],
 [ 7.51502991e+00,  2.87427902e+00,  3.03167343e+00],
 [ 1.07150269e+01,  7.54280090e-01,  2.56168365e+00],
 [ 1.07850266e+01, -8.75720978e-01, -7.88326263e-01],
 [ 9.63500977e+00, -3.33572388e+00, -3.63832474e+00],
 [ 1.22050095e+01, -4.61572647e+00, -6.53831482e+00],
 [ 1.56250000e+01, -4.05572891e+00, -7.67831802e+00],
 [ 1.73350143e+01, -1.95571899e+00, -1.04583168e+01],
 [ 1.43650055e+01, -1.00572586e+00, -1.26583290e+01],
 [ 1.14550018e+01,  1.25427246e+00, -1.23983192e+01],
 [ 1.04850006e+01,  4.01428223e+00, -9.62831497e+00],
 [ 7.44503021e+00,  5.43428421e+00, -8.86831284e+00],
 [ 4.07501221e+00,  4.42428589e+00, -7.68831253e+00],
 [ 2.47501373e+00,  8.04271698e-01, -7.70832062e+00],
 [ 3.27503204e+00,  7.64274597e-01, -1.15083313e+01],
 [ 1.77501678e+00, -2.90573120e+00, -1.23883247e+01],
 [ 5.35011292e-01, -5.96572876e+00, -1.03083229e+01],
 [-1.65496826e+00, -4.59571838e+00, -7.34832764e+00],
 [-4.29496765e+00, -1.80572510e+00, -7.55832672e+00],
 [-2.98500061e+00,  1.44428253e+00, -5.64832687e+00],
 [-2.44499207e+00,  4.70428467e+00, -3.23831558e+00],
 [-3.16497803e+00,  2.40427017e+00, -2.48317719e-01],
 [-6.27498627e+00,  3.24268341e-01, -9.08317566e-01],
 [-9.46498871e+00,  1.16428375e+00,  9.71679688e-01],
 [-1.08250046e+01, -1.99571609e+00,  2.84168243e+00],
 [-9.03499603e+00, -5.37572861e+00,  2.76166916e+00],
 [-9.74498749e+00, -8.41571808e+00,  4.70167160e+00],
 [-1.10249710e+01, -1.20457268e+01,  4.55167770e+00],
 [-1.35049820e+01, -1.30857277e+01,  1.80168152e+00],
 [-1.56449890e+01, -1.51557159e+01,  4.24167633e+00],
 [-1.93949890e+01, -1.53457260e+01,  3.60168457e+00],
 [-2.30349731e+01, -1.47157288e+01,  1.93168640e+00],
 [-2.25249863e+01, -1.50957298e+01, -2.02831268e+00],
 [-2.11249924e+01, -1.61757278e+01, -5.49831390e+00],
 [-2.02549744e+01, -1.44457245e+01, -9.14833069e+00],
 [-1.87649918e+01, -1.62057304e+01, -1.19283295e+01],
 [-2.00650024e+01, -1.94457245e+01, -1.34683228e+01],
 [-1.99950027e+01, -2.19257164e+01, -1.66083145e+01],
 [-2.16049957e+01, -2.49257278e+01, -1.76483154e+01],
 [-2.31449890e+01, -2.55457306e+01, -2.09883308e+01]]

In [18]:
import hoomd
import numpy as np
from hoomd import md

particles = {'A': {'id': 1, 'name': 'ALA', 'mass': 71.0800018, 'q': 0.0, 'lambda': 0.72973, 'sigma': 5.04}, 'R': {'id': 2, 'name': 'ARG', 'mass': 156.10111, 'q': 1.0, 'lambda': 0.0, 'sigma': 6.56}, 'N': {'id': 3, 'name': 'ASN', 'mass': 114.099998, 'q': 0.0, 'lambda': 0.432432, 'sigma': 5.68}, 'D': {'id': 4, 'name': 'ASP', 'mass': 115.099998, 'q': -1.0, 'lambda': 0.378378, 'sigma': 5.58}, 'C': {'id': 5, 'name': 'CYS', 'mass': 103.00919, 'q': 0.0, 'lambda': 0.594595, 'sigma': 5.48}, 'E': {'id': 6, 'name': 'GLU', 'mass': 129.100006, 'q': -1.0, 'lambda': 0.459459, 'sigma': 5.92}, 'Q': {'id': 7, 'name': 'GLN', 'mass': 128.100006, 'q': 0.0, 'lambda': 0.513514, 'sigma': 6.02}, 'G': {'id': 8, 'name': 'GLY', 'mass': 57.0499992, 'q': 0.0, 'lambda': 0.648649, 'sigma': 4.5}, 'H': {'id': 9, 'name': 'HIS', 'mass': 137.100006, 'q': 0.5, 'lambda': 0.513514, 'sigma': 6.08}, 'I': {'id': 10, 'name': 'ILE', 'mass': 113.199997, 'q': 0.0, 'lambda': 0.972973, 'sigma': 6.18}, 'L': {'id': 11, 'name': 'LEU', 'mass': 113.199997, 'q': 0.0, 'lambda': 0.972973, 'sigma': 6.18}, 'K': {'id': 12, 'name': 'LYS', 'mass': 128.199997, 'q': 1.0, 'lambda': 0.513514, 'sigma': 6.36}, 'M': {'id': 13, 'name': 'MET', 'mass': 131.199997, 'q': 0.0, 'lambda': 0.837838, 'sigma': 6.18}, 'F': {'id': 14, 'name': 'PHE', 'mass': 147.199997, 'q': 0.0, 'lambda': 1.0, 'sigma': 6.36}, 'P': {'id': 15, 'name': 'PRO', 'mass': 97.1200027, 'q': 0.0, 'lambda': 1.0, 'sigma': 5.56}, 'S': {'id': 16, 'name': 'SER', 'mass': 87.0800018, 'q': 0.0, 'lambda': 0.594595, 'sigma': 5.18}, 'T': {'id': 17, 'name': 'THR', 'mass': 101.099998, 'q': 0.0, 'lambda': 0.675676, 'sigma': 5.62}, 'W': {'id': 18, 'name': 'TRP', 'mass': 186.07931, 'q': 0.0, 'lambda': 0.945946, 'sigma': 6.78}, 'Y': {'id': 19, 'name': 'TYR', 'mass': 163.199997, 'q': 0.0, 'lambda': 0.864865, 'sigma': 6.46}, 'V': {'id': 20, 'name': 'VAL', 'mass': 99.0699997, 'q': 0.0, 'lambda': 0.891892, 'sigma': 5.86}}
particle_types = list(particles.keys())
l = 2000
seq     = 'MGDYGFGVLVQSNTGNKSAFPVRFHPHLQPPHHHQNATPSPAAFINNNTAANGSSAGSAWLFPAPATHNIQDEILGSEKAKSQQQEQQDPLEKQQLSPSPGQEAGILPETEKAKSEENQGDNSSENGNGKEKIRIESPVLTGFDYQEATGLGTSTQPLTSSASSLTGFSNWSAAIAPSSSTIINEDASFFHQGGVPAASANNGALLFQNFPHHVSPGFGGSFSPQIGPLSQHHPHHPHFQHHHSQHQQQRRSPASPHPPPFTHRNAAFNQLPHLANNLNKPPSPWSSYQSPSPTPSSSWSPGGGGYGGWGGSQGRDHRRGLNGGITPLNSISPLKKNFASNHIQLQKYARPSSAFAPKSWMEDSLNRADNIFPFPDRPRTFDMHSLESSLIDIMRAENDTIKARTYGRRRGQSSLFPMEDGFLDDGRGDQPLHSGLGSPHCFSHQNGE'

hoomd.context.initialize("")

box = hoomd.data.boxdim(Lx=l, Ly=l, Lz=l)
snap = hoomd.data.make_snapshot(N=len(seq),
                                box=box,
                                bond_types=['harmonic'],
                                particle_types=particle_types)

snap.particles.position[:] = pos

bond_arr = []
for i, aa in enumerate(seq):
    snap.particles.typeid[i]=particles[aa]["id"]
    bond_arr.append([i,i+1])
del bond_arr[-1]
snap.bonds.resize(len(seq)-1);
snap.bonds.group[:] = bond_arr[:]

hoomd.init.read_snapshot(snap);

harmonic = md.bond.harmonic()
harmonic.bond_coeff.set('harmonic', k=400.0, r0=3.8)


nl = md.nlist.cell()
lj = hoomd.md.pair.lj(r_cut=3.0, nlist=nl)

# debye = md.charge.pppm(group='all', nlist=nl)
# hps_table = md.pair.table(width=len(seq), nlist=nl);
for i in range(len(particle_types)):
    aa_i = particle_types[i]
    for j in range(i, len(particle_types)):
        aa_j = particle_types[j]
#         print(aa_i, aa_j)
        lj.pair_coeff.set(str(aa_i), str(aa_j), epsilon=1.0, sigma=1.0)
#         lj.pair_coeff.set('A', 'A', epsilon=1.0, sigma=1.0)

        
#         lambd = (particles[aa_i]["lambda"] + particles[aa_j]["lambda"])/2
#         sigma = (particles[aa_i]["sigma"] + particles[aa_j]["sigma"])/2
#         hps_table.pair_coeff.set(aa_i, aa_j, func=HPS, 
#                                  rmin=0.8, 
#                                  rmax=0.9,
#                                  coeff=dict(eps=10, lambd=lambd, sigma=4))


# lj = md.pair.lj(r_cut=3.0, nlist=nl)
# lj.pair_coeff.set('A', 'A', epsilon=1.0, sigma=1.0)

# Integrate at constant temperature
md.integrate.mode_standard(dt=0.005)
hoomd.md.integrate.langevin(group=hoomd.group.all(), kT=1.2, seed=4)

# Run for 10,00 time steps
hoomd.run(10e1)

notice(2): Group "all" created containing 448 particles
notice(2): integrate.langevin/bd is using specified gamma values
notice(2): -- Neighborlist exclusion statistics -- :
notice(2): Particles with 1 exclusions             : 2
notice(2): Particles with 2 exclusions             : 446
notice(2): Neighbors included by diameter          : no
notice(2): Neighbors excluded when in the same body: no
** starting run **


**ERROR**: out of memory before /hoomd/GPUArray.h:947


RuntimeError: CUDA Error